In [5]:
import numpy as np
import torch
from scipy.integrate import solve_ivp
import torch.nn as nn

In [70]:
class ChaoticLayer(nn.Module):
    def __init__(self,iterations=1000,dt=0.01):
        super(ChaoticLayer,self).__init__()
        self.iterations = iterations
        self.dt = dt
    def forward(self,x):
        def lorenz(t,state, beta = 8/3, rho = 28, sigma = 10):
            x,y,z = state
            dx = sigma*(y-x)
            dy = rho*x-y-x*z
            dz = -beta*z + x*y
            return [dx,dy,dz]
        trajectories = []
        for sample in x:
            x1 = sample[0].item()
            x2 = sample[1].item()
            initial_state = [x1,x2,np.sqrt(x1**2+x2**2)]
            times_at_which_func_eval = np.linspace(0,self.iterations*self.dt,self.iterations)
            trajectory = solve_ivp(fun = lorenz,
                                  y0 = initial_state,
                                  t_span = [0,self.iterations*self.dt],
                                  t_eval = times_at_which_func_eval,
                                  )
            '''
            We only take the second half of the trajectory since the Lorenz attractor needs some 
            time to stabilise before exhibiting chaotic behaviour
            '''
            halfway_point = trajectory.y.shape[1] // 2
            x = trajectory.y[0,halfway_point:]
            y = trajectory.y[1,halfway_point:]
            z = trajectory.y[2,halfway_point:]
            # we take the summary statistics of all 3-D points to get the features we are gonna use for our prediction
            x_mean = np.mean(x)
            y_mean = np.mean(y)
            z_mean = np.mean(z)
            z_std_dev = np.std(z)
            y_std_dev = np.std(y)
            x_std_dev = np.std(x)
            corr = np.mean(np.sqrt(x**2+y**2+z**2))
            features_of_trajectory = [x_mean,y_mean,z_mean,x_std_dev,y_std_dev,z_std_dev,corr]
            trajectories.append(features_of_trajectory)
        return torch.tensor(trajectories,dtype = torch.float32)

In [71]:
class MainNet(nn.Module):
    def __init__(self):
        super(MainNet,self).__init__()
        self.input_fc = nn.Linear(2,16)
        self.input_bn = nn.BatchNorm1d(16)

        self.chaotic_layer = ChaoticLayer(iterations=1000,dt=0.01)
        
        self.hidden = nn.Linear(16+7,16)
        self.hidden_bn = nn.BatchNorm1d(16)

        self.output = nn.Linear(16,1)
        self.relu = nn.ReLU()
    def forward(self,x):
        x_hidden = self.input_fc(x)
        x_hidden = self.input_bn(self.relu(x_hidden))
        x_chaotic = self.chaotic_layer(x)
        x_combined = torch.cat([x_hidden,x_chaotic],dim=1)
        x_combined = self.hidden(x_combined)
        x_combined = self.hidden_bn(x_combined)
        x_combined = self.relu(x_combined)
        x_out = self.output(x_combined)
        return x_out

In [63]:
def computeLCM(x,y):
    x,y = int(x),int(y)
    return abs(x*y)//np.gcd(x,y)

In [64]:
from sklearn.preprocessing import MinMaxScaler

In [79]:
inputs = torch.randint(1,50,(200,2)).float()
target = torch.tensor([computeLCM(a,b) for a,b in inputs.numpy()],dtype=torch.float32).unsqueeze(1)
input_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
inputs = torch.tensor(input_scaler.fit_transform(inputs),dtype=torch.float32)
target = torch.tensor(target_scaler.fit_transform(target),dtype=torch.float32)

train_size = 160
input_train,input_test = inputs[:train_size],inputs[train_size:]
target_train,target_test = target[:train_size],target[train_size:]

In [66]:
from torch.optim import RAdam

In [74]:
model = MainNet()
optimiser = RAdam(model.parameters(),lr=0.005)
criterion = nn.MSELoss()
for epoch in range(1000):
    model.train()
    optimiser.zero_grad()
    outputs = model(input_train)
    loss = criterion(outputs,target_train)

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    loss.backward()
    optimiser.step()
    if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.2299
Epoch 50, Loss: 0.0222
Epoch 100, Loss: 0.0119
Epoch 150, Loss: 0.0109
Epoch 200, Loss: 0.0105
Epoch 250, Loss: 0.0102
Epoch 300, Loss: 0.0098
Epoch 350, Loss: 0.0094
Epoch 400, Loss: 0.0091
Epoch 450, Loss: 0.0088
Epoch 500, Loss: 0.0085
Epoch 550, Loss: 0.0083
Epoch 600, Loss: 0.0080
Epoch 650, Loss: 0.0076
Epoch 700, Loss: 0.0072
Epoch 750, Loss: 0.0070
Epoch 800, Loss: 0.0067
Epoch 850, Loss: 0.0066
Epoch 900, Loss: 0.0064
Epoch 950, Loss: 0.0062


In [82]:
    model.eval()
    with torch.no_grad():
        test_outputs = model(input_test)
        test_loss = criterion(test_outputs, target_test)
        print(f"Test Loss: {test_loss.item():.4f}")

        # Inverse transform
        test_outputs_orig = target_scaler.inverse_transform(test_outputs.numpy())
        test_lcm_targets_orig = target_scaler.inverse_transform(target_test.numpy())

        print("\nSample Predictions (Original Scale):")
        for i in range(min(10, len(input_test))):
            print(
                f"Input: {input_test[i].numpy()}, "
                f"Predicted LCM: {test_outputs_orig[i][0]:.2f}, "
                f"True LCM: {test_lcm_targets_orig[i][0]}"
            )

        # MAPE
        mape = torch.mean(
            torch.abs(
                (torch.tensor(test_lcm_targets_orig) - torch.tensor(test_outputs_orig))
                / torch.tensor(test_lcm_targets_orig)
            )
        ) * 100
        print(f"\nMean Absolute Percentage Error (MAPE): {mape.item():.2f}%")

Test Loss: 0.0513

Sample Predictions (Original Scale):
Input: [0.6041667 0.9791667], Predicted LCM: 1190.36, True LCM: 239.99998474121094
Input: [0.7291667  0.16666667], Predicted LCM: 201.68, True LCM: 36.0
Input: [0.0625     0.41666666], Predicted LCM: -224.53, True LCM: 84.00000762939453
Input: [0.6458333 0.7083333], Predicted LCM: 1189.26, True LCM: 1120.0
Input: [0.75       0.41666666], Predicted LCM: 916.66, True LCM: 777.0
Input: [0.9375    0.9166667], Predicted LCM: 2663.95, True LCM: 2070.0
Input: [0.39583334 1.        ], Predicted LCM: 1091.99, True LCM: 980.0
Input: [0.39583334 0.0625    ], Predicted LCM: 94.05, True LCM: 20.0
Input: [0.7291667  0.14583333], Predicted LCM: 90.94, True LCM: 72.0
Input: [0.75 0.5 ], Predicted LCM: 898.16, True LCM: 925.0

Mean Absolute Percentage Error (MAPE): 348.54%
